# 遊戯王データセット カード画像ダウンロード
# Yu-Gi-Oh Data Set Extender

カードに紐づけられたデータを増やします  
ディレクトリは```../extend_input/```下に配置されます

現状
* カラムとパラメータを設定し画像を```../extend_input/card_images/```に配置します、ファイル名は拡張子を除いてカード名と同一です
* ~~カード効果を取得します~~



Increase the data linked to the card  
The directory is placed under ```../ extend_input /```


Current status
* Set the column and parameters and place the image in ```../ extend_input / card_images /```, the file name is the same as the card name except for the extension
* ~~Get card effects~~

In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pprint import pprint as pp

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import sys
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['card_data.csv']


In [16]:
# please set your email adress.
ADRESS = ''

ROBOT_NAME = 'Kaggle yu-gi-oh Crawler ({})'
CRAWRING_INTERVAL = 0.1
SAVE_EXT = '.jpg'

## カードのパラメータを辞書型で全取得(atk, def以外)
## Get all card parameters in dictionary type (except atk and def)

In [17]:
cards_data_csv = os.path.join('../input', 'card_data.csv')
df = pd.read_csv(cards_data_csv, index_col=0)
cols = df.columns.values
card_names = df.index.values
card_type_dir = {}
for col in cols:
    if col == 'ATK' or col == 'DEF':
        continue
    if not col in card_type_dir:
        card_type_dir[col] = []
    for status in df[col]:
        if not status in card_type_dir[col] and not pd.isnull(status):
            card_type_dir[col].append(status)


pp(cols)
pp(card_type_dir)
for param in card_type_dir.keys():
    print('{} has {} elements.'.format(param, str(len(card_type_dir[param]))))

array(['Type', 'Level', 'Race', 'Attribute', 'ATK', 'DEF'], dtype=object)
{'Attribute': ['DARK', 'FIRE', 'EARTH', 'WIND', 'LIGHT', 'WATER', 'DIVINE'],
 'Level': [3.0, 4.0, 8.0, 6.0, 5.0, 2.0, 1.0, 10.0, 9.0, 12.0, 7.0, 11.0],
 'Race': ['Continuous',
          'Zombie',
          'Normal',
          'Fiend',
          'Quick-Play',
          'Rock',
          'Warrior',
          'Winged Beast',
          'Spellcaster',
          'Beast',
          'Fairy',
          'Equip',
          'Field',
          'Fish',
          'Beast-Warrior',
          'Thunder',
          'Machine',
          'Sea Serpent',
          'Aqua',
          'Plant',
          'Dragon',
          'Reptile',
          'Counter',
          'Psychic',
          'Insect',
          'Pyro',
          'Dinosaur',
          'Wyrm',
          'Ritual',
          'Divine-Beast',
          'Cyberse'],
 'Type': ['Trap Card',
          'Normal Monster',
          'Flip Effect Monster',
          'Effect Monster',
          '

## 上のパラメータ一覧から欲しい物をaim_targetに入れる
## Add the desired item from the above parameter list to aim_target

In [18]:
aim_target = {
    'Type': [
        'Normal Monster',
        'Effect Monster'
    ]
}

## aim_targetに基づいてゾーニング
今回はテストです
```
'Type': [
        'Normal Monster',
        'Effect Monster'
    ]
```
を収集しています  
## Zoning based on aim_target
This time is a test
```
'Type': [
         'Normal Monster',
         'Effect Monster'
     ]
```
Collecting

In [19]:
target_card_names = []
if len(aim_target) != 0:
    for param in aim_target.keys():
        for element in aim_target[param]:
            target_card_names.append(df[(df[param] == element)].index.values)
    target_card_names = list(set([flat for inner in target_card_names for flat in inner]))
else:
    target_card_names = card_names
print('crawling card num :{}'.format(len(target_card_names)))

del card_names

crawling card num :2982


## データを拡張します
## Expand the data

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.parse import quote
from mimetypes import guess_extension
from time import sleep
from tqdm import tqdm_notebook

os.makedirs(os.path.join('../extend_input', 'card_images'), exist_ok=True)
# print(os.listdir('..'))
# print(quote('たのしい ピクニック'))
url = 'https://www.bing.com/images/search?q='
img_link = []
header = {
    'User-Agent': ROBOT_NAME.format(ADRESS) +' Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36' 
}
for cname in tqdm_notebook(target_card_names):
    url_with_query = url + quote(cname) + "&FORM=HDRSC2"
    req = Request(url_with_query, headers=header)
    try:
        with urlopen(req, timeout=3) as p:
            content = p.read()
            mime = p.getheader('Content-Type')
            if not mime or not content:
                continue
    except:
        sys.stderr.write('Error in fetching :'+ format(url_with_query) + "\n")
        continue
    soup = BeautifulSoup(content, 'html.parser')
    img_elem = soup.find_all('div', attrs={'class': 'hoff'})[0].find('img' , attrs={'class': 'mimg'}).get('src').split('&')[0]
#     ext = guess_extension(mime.split(';')[0])
#     if ext in ('.jpeg', '.jpg', '.png', '.jpe'):
    ext = SAVE_EXT
    try:
        with open(os.path.join('../extend_input', 'card_images', cname + ext), 'wb') as f:
            f.write(urlopen(img_elem).read())
    except:
        sys.stderr.write('Error in Saving :'+ format(cname) + "\n")
        continue
        sys.exit(0)
    print('/saved. {}/'.format(cname + ext), end='')
    sleep(CRAWRING_INTERVAL)

/saved. Caligo Claw Crow.jpg//saved. Garvas.jpg//saved. Decayed Commander.jpg//saved. Volcanic Queen.jpg//saved. Damage Mage.jpg//saved. Wattpheasant.jpg//saved. Frost and Flame Dragon.jpg//saved. Mermail Abysspike.jpg//saved. Cyber Valley.jpg//saved. White Tiger Summoner.jpg//saved. Vampire Lord.jpg//saved. Digital Bug Centibit.jpg//saved. Pandemonium Watchbear.jpg//saved. Giant Orc.jpg//saved. Fluffal Cat.jpg//saved. Ghostrick Stein.jpg//saved. Nekogal 2.jpg//saved. Stone Armadiller.jpg//saved. Fenrir the Nordic Wolf.jpg//saved. Ljosalf of the Nordic Alfar.jpg//saved. Job-Change Mirror.jpg//saved. Phantom Dragon.jpg//saved. Constellar Siat.jpg//saved. Firestorm Prominence.jpg//saved. Nanobreaker.jpg//saved. Power Supplier.jpg//saved. Mist Valley Watcher.jpg//saved. Brotherhood of the Fire Fist - Gorilla.jpg//saved. Crystron Thystvern.jpg//saved. Headless Knight.jpg//saved. Gorz the Emissary of Darkness.jpg//saved. World of Prophecy.jpg//saved. Deskbot 003.jpg//saved. The Calculator.j

/saved. Command Knight.jpg//saved. Evilswarm Mandragora.jpg//saved. Master Kyonshee.jpg//saved. Reflect Bounder.jpg//saved. Cannon Soldier MK-2.jpg//saved. Armored Lizard.jpg//saved. Gilasaurus.jpg//saved. Marmiting Captain.jpg//saved. Ground Spider.jpg//saved. Cave Dragon.jpg//saved. Birdface.jpg//saved. Malefic Red-Eyes B. Dragon.jpg//saved. Infernity General.jpg//saved. Blue Flame Swordsman.jpg//saved. Petit Moth.jpg//saved. Gokipon.jpg//saved. The Little Swordsman of Aile.jpg//saved. Cybernetic Magician.jpg//saved. Hazy Flame Hyppogrif.jpg//saved. Fusilier Dragon, the Dual-Mode Beast.jpg//saved. Yubel - The Ultimate Nightmare.jpg//saved. Meltiel, Sage of the Sky.jpg//saved. Mermail Abyssleed.jpg//saved. Exarion Universe.jpg//saved. Deep-Space Cruiser IX.jpg//saved. Hieratic Dragon of Sutekh.jpg//saved. Fabled Grimro.jpg//saved. Naturia Dragonfly.jpg//saved. Vennominaga the Deity of Poisonous Snakes.jpg//saved. Evil HERO Malicious Edge.jpg//saved. Chronomaly Crystal Bones.jpg//saved

Error in fetching :https://www.bing.com/images/search?q=Spiritual%20Beast%20Pettlephin&FORM=HDRSC2


/saved. Fairywitch.jpg//saved. Hyena.jpg//saved. Hushed Psychic Cleric.jpg//saved. Infernity Randomizer.jpg//saved. Naturia Mosquito.jpg//saved. Dark Elf.jpg//saved. Bazoo the Soul-Eater.jpg//saved. Gishki Reliever.jpg//saved. Big Whale.jpg//saved. Blackwing - Boreas the Sharp.jpg//saved. Gravekeeper's Spear Soldier.jpg//saved. Inzektor Ladybug.jpg//saved. Galaxy Soldier.jpg//saved. Enraged Battle Ox.jpg//saved. U.A. Rival Rebounder.jpg//saved. Jumbo Drill.jpg//saved. ZW - Lightning Blade.jpg//saved. Mystical Shine Ball.jpg//saved. Rapid-Fire Magician.jpg//saved. Ally of Justice Cosmic Gateway.jpg//saved. Double Coston.jpg//saved. Metalzoa.jpg//saved. Saggi the Dark Clown.jpg//saved. Flame Champion.jpg//saved. Silent Swordsman LV3.jpg//saved. Vylon Ohm.jpg//saved. Fluffal Penguin.jpg//saved. Toy Knight.jpg//saved. Gladiator Beast Lanista.jpg//saved. Inzektor Dragonfly.jpg//saved. Reed Butterfly.jpg//saved. Gogogo Goram.jpg//saved. Leogun.jpg//saved. Karate Man.jpg//saved. Guardian Baou

/saved. Evolsaur Pelta.jpg//saved. Hyo.jpg//saved. Water Element.jpg//saved. Change Slime.jpg//saved. Superheavy Samurai Daihachi.jpg//saved. Samsara Kaiser.jpg//saved. Total Defense Shogun.jpg//saved. DZW - Chimera Clad.jpg//saved. Dragon Dowser.jpg//saved. Ruffian Railcar.jpg//saved. Armageddon Knight.jpg//saved. Sphere Kuriboh.jpg//saved. Hardened Armed Dragon.jpg//saved. Mermail Abyssbalaen.jpg//saved. Sage of Stillness.jpg//saved. Caravan of the Ice Barrier.jpg//saved. Vampire Lady.jpg//saved. Chronomaly Mud Golem.jpg//saved. Miscellaneousaurus.jpg//saved. Speed Warrior.jpg//saved. Embryonic Beast.jpg//saved. Swap Frog.jpg//saved. SPYRAL Super Agent.jpg//saved. Ghostrick Yuki-onna.jpg//saved. Alligator's Sword.jpg//saved. Raidraptor - Sharp Lanius.jpg//saved. Wattsquirrel.jpg//saved. Farfa, Malebranche of the Burning Abyss.jpg//saved. Igknight Lancer.jpg//saved. Zombowwow.jpg//saved. Union Rider.jpg//saved. Stronghold Guardian.jpg//saved. Wow Warrior.jpg//saved. Steelswarm Caller.

/saved. Anarchist Monk Ranshin.jpg//saved. Genesis Dragon.jpg//saved. Warrior Lady of the Wasteland.jpg//saved. Reptilianne Vaskii.jpg//saved. Gravekeeper's Chief.jpg//saved. Meklord Emperor Wisel.jpg//saved. Cross-Sword Beetle.jpg//saved. Throwstone Unit.jpg//saved. Evilswarm Ketos.jpg//saved. Flower Cardian Zebra Grass with Moon.jpg//saved. Cyber Dragon Core.jpg//saved. Hand-Holding Genie.jpg//saved. Light Effigy.jpg//saved. Archfiend Heiress.jpg//saved. Igknight Champion.jpg//saved. Majestic Mech - Ohka.jpg//saved. Empress Mantis.jpg//saved. Silent Angler.jpg//saved. Truckroid.jpg//saved. Hannibal Necromancer.jpg//saved. Elemental HERO Burstinatrix.jpg//saved. Garbage Lord.jpg//saved. SZW - Fenrir Sword.jpg//saved. Bull Blader.jpg//saved. Cursed Fig.jpg//saved. Garum the Storm Vassal.jpg//saved. Destiny HERO - Drilldark.jpg//saved. Familiar-Possessed - Aussa.jpg//saved. Performapal Springoose.jpg//saved. Cyberdark Edge.jpg//saved. Madjinn Gunn.jpg//saved. Gem-Knight Emerald.jpg//sav

/saved. Silent Magician LV4.jpg//saved. Genex Furnace.jpg//saved. Mystical Capture Chain.jpg//saved. Cockroach Knight.jpg//saved. Flelf.jpg//saved. Koa'ki Meiru Crusader.jpg//saved. Vice Dragon.jpg//saved. Satellarknight Vega.jpg//saved. Cyber Prima.jpg//saved. Dwarf Star Dragon Planeter.jpg//saved. Kabazauls.jpg//saved. Wind-Up Snail.jpg//saved. Superheavy Samurai Soulclaw.jpg//saved. Brotherhood of the Fire Fist - Rooster.jpg//saved. Sky Scourge Enrise.jpg//saved. Playful Possum.jpg//saved. Hyper-Ancient Shark Megalodon.jpg//saved. Panther Warrior.jpg//saved. Deepsea Warrior.jpg//saved. Mecha Phantom Beast Hamstrat.jpg//saved. Diabound Kernel.jpg//saved. Destiny HERO - Diamond Dude.jpg//saved. Master Monk.jpg//saved. Chronomaly Gordian Knot.jpg//saved. XX-Saber Ragigura.jpg//saved. Inari Fire.jpg//saved. Madolche Messengelato.jpg//saved. Flying Penguin.jpg//saved. Sharkraken.jpg//saved. Galaxy Wizard.jpg//saved. White Night Queen.jpg//saved. Firewing Pegasus.jpg//saved. Gaia Soul the

/saved. Arlownay.jpg//saved. Interplanetarypurplythorny Dragon.jpg//saved. Hero Kid.jpg//saved. The Tricky.jpg//saved. Raidraptor - Skull Eagle.jpg//saved. Risebell the Star Adjuster.jpg//saved. Spherous Lady.jpg//saved. Soul Drain Dragon.jpg//saved. Bird of Roses.jpg//saved. Geira Guile the Cubic King.jpg//saved. Etoile Cyber.jpg//saved. Imprisoned Queen Archfiend.jpg//saved. PSY-Frame Driver.jpg//saved. Infernoid Seitsemas.jpg//saved. Morphtronic Vacuumen.jpg//saved. The Creator Incarnate.jpg//saved. Mokey Mokey.jpg//saved. Noble Knight Bedwyr.jpg//saved. Tenkabito Shien.jpg//saved. Yaranzo.jpg//saved. The Black Stone of Legend.jpg//saved. Gradius' Option.jpg//saved. Garoth, Lightsworn Warrior.jpg//saved. Naturia Mantis.jpg//saved. Ganbara Knight.jpg//saved. Drillroid.jpg//saved. Ally of Justice Quarantine.jpg//saved. ZW - Unicorn Spear.jpg//saved. Cross Porter.jpg//saved. Root Water.jpg//saved. Jetroid.jpg//saved. Djinn Cursenchanter of Rituals.jpg//saved. Legendary Maju Garzett.jpg

In [ ]:
# url = 'https://www.bing.com/images/search?q='+quote('Gravekeepers Watcher')+"&FORM=HDRSC2"
# req = Request(url, headers=header)
# try:
#     with urlopen(req, timeout=3) as p:
#         content = p.read()
#         mime = p.getheader('Content-Type')
#         if not mime or not content:
#             pass
# except:
#     sys.stderr.write('Error in fetching :'+ format(url) + "\n")
# soup = BeautifulSoup(content, 'lxml')
# img_elem = soup.find_all('div', attrs={'class': ['img_cont', 'hoff']})[0].find('img' , attrs={'class': 'mimg'})
# print(img_elem.get('src').split('&')[0])

I will continue to enhance the functions.  
Thanks a lot 😆